In [1]:
# 导入OpenAI库，并创建OpenAI客户端
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()

In [2]:
# 导入数据文件
import pandas as pd
file_path = 'Spotify_Songs.csv'
sales_data = pd.read_csv(file_path, nrows=20)
sales_data.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [3]:
# 创建文件
file = client.files.create(
  file=open(file_path, "rb"),
  purpose='assistants',
)

# 创建一个包含这个文件的助手
assistant = client.beta.assistants.create(
  instructions="作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。",
  model="gpt-4-0125-preview",
  tools=[
    {"type": "code_interpreter"}
  ],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]  # Here we add the file id
    }
  }
)
print(assistant)

Assistant(id='asst_frwRB2Ugjj2XVUGnRy5f3no7', created_at=1717210909, description=None, instructions='作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。', metadata={}, model='gpt-4-0125-preview', name=None, object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-TgNxU9yP74IectmQt2ScOYhY']), file_search=None), top_p=1.0, temperature=1.0, response_format='auto')


In [12]:
# 创建对话线程并运行
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "根据数据集的内容，选择最有代表性的字段，根据畅销音乐的特点和趋势做一张可视化表格，每个音乐家用不同的颜色表示。",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [
            { "type": "code_interpreter" }
          ]
        }
      ]
    }
  ]
)
print(thread)

Thread(id='thread_iIvaQeLDBTSUa6Zacil8KQ80', created_at=1717223262, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-TgNxU9yP74IectmQt2ScOYhY']), file_search=None))


In [13]:
# 创建Run来运行和助手的对话
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
print(run)

Run(id='run_EDjLmAUQ6PHRxgKHCUNkBk1h', assistant_id='asst_frwRB2Ugjj2XVUGnRy5f3no7', cancelled_at=None, completed_at=None, created_at=1717223264, expires_at=1717223864, failed_at=None, incomplete_details=None, instructions='作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_iIvaQeLDBTSUa6Zacil8KQ80', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [14]:
# 检查并等待可视化完成
import time
while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        # 检查是否创建了图像
        messages.data[0].content[0].image_file
        # 等待运行完成
        time.sleep(5)
        print('图表已创建！')
        if messages.data and messages.data[0].content:
            print('当前Message:', messages.data[0].content[0])
        break
    except:
        time.sleep(10)
        print('您的Assistant正在努力做图表呢...')
        if messages.data and messages.data[0].content:
            print('当前Message:', messages.data[0].content[0])

您的Assistant正在努力做图表呢...
您的Assistant正在努力做图表呢...
您的Assistant正在努力做图表呢...
当前Message: TextContentBlock(text=Text(annotations=[], value='根据数据集的内容，我们有以下字段可以用来表示畅销音乐的特点和趀势：\n\n- `track_name`: 音轨名称\n- `artist(s)_name`: 艺术家名称\n- `artist_count`: 艺术家数量\n- `released_year`: 发布年份\n- `streams`: 流量\n- `bpm` (Beats Per Minute): 节拍速度\n- `key`: 调性\n- `mode`: 调式\n- `danceability_%`: 可跳舞性百分比\n- `valence_%`: 歌曲情感的积极性百分比\n- `energy_%`: 能量百分比\n- `acousticness_%`: 声学性百分比\n- `instrumentalness_%`: 乐器音乐性百分比\n- `liveness_%`: 现场感百分比\n- `speechiness_%`: 语音性百分比\n\n考虑到要根据畅销音乐的特点和趋势进行可视化，同时也要每个音乐家以不同的颜色表示，我们可以选择几个关键特征如`streams` (流量)、`danceability_%` (可跳舞性)和`energy_%`（能量）来进行可视化，并以`released_year`为X轴，以展现趋势。\n\n接下来，我将会创建一张图表，展示不同艺术家的歌曲流量、可跳舞性和能量随时间的变化趋势。由于艺术家数量可能较多，为了可视化的清晰度，我们可能需要限制显示的艺术家数量。现在让我们先检查一下有多少位不同的艺术家，并确定一个合理的显示方法。'), type='text')
您的Assistant正在努力做图表呢...
当前Message: TextContentBlock(text=Text(annotations=[], value='数据集包含645位不同的艺术家，这意呩着如果我们尝试将所有艺术家的数据都展示在一张图表上，图表可能会非常混乱且难以阅读。因此，为了创建一个清晰且易于解读的可视化，我们将选择几位流量最高的艺术家进行展示。\

In [15]:
# 转换输出文件为PNG格式
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)

plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "音乐趋势图.png"
convert_file_to_png(plot_file_id,image_path)

# 上传图表
plot_file = client.files.create(
  file=open(image_path, "rb"),
  purpose='assistants'
)

In [16]:
# 看看助手整个的思维和行动过程
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]

[ImageFileContentBlock(image_file=ImageFile(file_id='file-03gX6k5HtwwlBQg6M75cRLe2'), type='image_file'),
 TextContentBlock(text=Text(annotations=[], value='我在尝试运行可视化代码时未能成功导入必要的可视化库`matplotlib`和`seaborn`。我会立即纠正这个问题并重试创建图表。让我们重新导入这些库，并绘制关于畅销歌曲的跳舞性随发布年份的变化图表。'), type='text'),
 TextContentBlock(text=Text(annotations=[], value='似乎在尝试清理数据和准备可视化时出现了一个语法错误。我会立即更正这个问题并重新尝试绘制图表。'), type='text'),
 TextContentBlock(text=Text(annotations=[], value='选择了流量最高的10首歌曲进行分析和可视化时，我们注意到一首歌曲"Love Grows (Where My Rosemary Goes)"的流量数据似乎受到了异常值的影响，并且它的发布年份为1970，这可能是一个占位或错误的值。因此，在接下来的可视化中，我们将排除这首歌曲。\n\n我们将基于以下字段进行可视化：\n- `track_name`：歌曲名称\n- `artist(s)_name`：艺术家名称\n- `released_year`：发布年份\n- `streams`：流量\n- `danceability_%`：可跳舞性百分比\n- `energy_%`：能量百分比\n\n每首歌曲将使用一个不同的颜色表示。由于原始数据的一些问题，我们需要进行适当的调整和清理。现在，我将进行这些步骤，并创建一张展示这些畅销歌曲随发布年份变化的可跳舞性和能量的趋势图。'), type='text'),
 TextContentBlock(text=Text(annotations=[], value='在选择前10位流量最高的艺术家时，出现了一些意外的结果，这可能是由于原始数据或处理过程中的错误。似乎有一些艺术家的名称被错误地整合成了其歌曲的特征描述或串连了多个数值，从而导致了异常的艺术家名称。\n\n为了绕

In [17]:
# 定义提交用户的消息的函数
def submit_message_wait_completion(assistant_id, thread, user_message, file_ids=None):
    # 检查并等待活跃的Run完成
    for run in client.beta.threads.runs.list(thread_id=thread.id).data:
        if run.status == 'in_progress':
            print(f"等待Run {run.id} 完成...")
            while True:
                run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status
                if run_status in ['succeeded', 'failed']:
                    break
                time.sleep(5)  # 等待5秒后再次检查状态

    # 提交消息
    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }
    # 设置attachments
    if file_ids:        
        attachments = [{"file_id": file_id, "tools": [ {"type": "code_interpreter"}]} for file_id in file_ids]
        params['attachments'] = attachments
    client.beta.threads.messages.create(**params)

    # 创建Run
    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
    return run 


In [18]:
# 使用submit_message函数, 来发送请求，让助手生成洞察
submit_message_wait_completion(assistant.id, thread, "请根据你刚才创建的图表，给我两个约20字的句子，描述最重要的洞察。这将用于幻灯片展示，揭示出数据背后的'秘密'。")

Run(id='run_WdgdIFOyQXTM9ATigdrKgham', assistant_id='asst_frwRB2Ugjj2XVUGnRy5f3no7', cancelled_at=None, completed_at=None, created_at=1717223943, expires_at=1717224543, failed_at=None, incomplete_details=None, instructions='作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_iIvaQeLDBTSUa6Zacil8KQ80', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [19]:
# 获取对话线程的响应
def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)

# 等待响应并打印洞察
time.sleep(10) # 假设助理需要一些时间来生成洞察
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)

1. "2019至2023年间，畅销歌曲的跳舞性明显上升，展现了流行音乐向更加活跃风格的转变。"
2. "Taylor Swift和Duncan Laurence的歌曲以超高流量领跑，彰显了其在全球音乐市场的巨大影响力。"


In [21]:
# 根据见解生成标题 
submit_message_wait_completion(assistant.id, thread, "根据你创建的情节和要点，为幻灯片想一个非常简短的标题。它应该只反映你得出的主要见解。")


Run(id='run_Kerm8qIBzG3zbRWkc79n0TN7', assistant_id='asst_frwRB2Ugjj2XVUGnRy5f3no7', cancelled_at=None, completed_at=None, created_at=1717224082, expires_at=1717224682, failed_at=None, incomplete_details=None, instructions='作为一名数据科学助理，当给定数据和一个查询时，你能编写适当的代码并创建适当的可视化。', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_iIvaQeLDBTSUa6Zacil8KQ80', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [22]:
# 等待响应并打印标题
time.sleep(10) # 等待助理生成标题
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)

"流行音乐趋势：活跃风格与巨星影响力"


In [23]:
# 提供我们公司的说明
company_summary = "我们是CD批发商，但是我们董事长也是个流行歌手！"

# 使用DALL-E 3生成图片
response = client.images.generate(
  model='dall-e-3',
  prompt=f"根据这个公司概述 {company_summary}, \
           创建一张展示成长和前进道路的启发性照片。这将用于季度销售规划会议",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url

# 获取DALL-E 3生成的图片
import requests
dalle_img_path = '音乐趋势图.png'
img = requests.get(image_url)

# 将图片保存到本地
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

# 上传图片提供给助手做为PPT素材
dalle_file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)

In [24]:
title_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# 创建新的演示文稿对象
prs = Presentation()

# 添加一个空白的幻灯片布局
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# 将幻灯片的背景颜色设置为黑色
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# 在幻灯片左侧添加图片，上下留有边距
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# 在较高位置添加标题文本框
left = prs.slide_width * 3/5
top = Inches(2)
width = prs.slide_width * 2/5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(38)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# 添加副标题文本框
left = prs.slide_width * 3/5
top = Inches(3)
width = prs.slide_width * 2/5
height = Inches(1)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_p = subtitle_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(22)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
"""

data_vis_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# 创建新的演示文稿对象
prs = Presentation()

# 添加一个空白的幻灯片布局
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# 将幻灯片的背景颜色设置为黑色
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# 定义占位符
image_path = data_vis_img
title_text = "提升利润：在线销售与直销优化的主导地位"
bullet_points = "• 在线销售在各个季度中始终领先于盈利能力，表明了强大的数字市场存在。\n• 直销表现出波动，表明该渠道的表现变化和需要针对性改进的必要性。"

# 在幻灯片左侧添加图片占位符
left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# 添加覆盖整个宽度的标题文本
left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# 添加硬编码的“关键见解”文本和项目符号列表
left = prs.slide_width * 2/3
top = Inches(1.5)
width = prs.slide_width * 1/3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "关键见解："
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()


bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""

title_text = "鸟语乐境"
subtitle_text = "2025年销售大会"

submit_message_wait_completion(assistant.id,thread,f"使用包含的代码模板创建符合模板格式的PPTX幻灯片，但使用本消息中包含的图片、公司名称/标题和文件名/副标题：\
{title_template}。重要提示：在此第一张幻灯片中使用本消息中包含的图片文件作为image_path图像，并使用公司名称 {title_text} 作为title_text变量，\
  使用副标题文本 {subtitle_text} 作为subtitle_text变量。\
    接着，使用以下代码模板创建第二张幻灯片：{data_vis_template}，创建符合模板格式的PPTX幻灯片，但使用公司名称/标题和文件名/副标题：\
{data_vis_template}。重要提示：使用您之前在本线程中创建的第二个附图（折线图）作为data_vis_img图像，并使用您之前创建的数据可视化标题作为title_text变量，\
  使用您之前创建的见解项目符号列表作为bullet_points变量。将这两张幻灯片输出为.pptx文件。确保输出为两张幻灯片，每张幻灯片都符合本消息中给出的相应模板。",
              file_ids=[dalle_file.id, plot_file.id]
)


# 等待助手完成PPT创建任务
while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("成功检索到 pptx_id:", pptx_id)
        break
    except Exception as e:
        print("您的Assistant正在努力制作幻灯片...")
        time.sleep(10)

import io
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file= client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("销售大会.pptx", "wb") as f:
    f.write(file_obj.getbuffer())


您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
您的Assistant正在努力制作幻灯片...
成功检索到 pptx_id: file-c39pmCGSIhl2llr1835lTMgP
